In [5]:
from mtcnn import MTCNN
from PIL import Image
import numpy as np

detector = MTCNN()

def extract_face(image_path, required_size=(160, 160)):
    image = Image.open(image_path)
    image = image.convert('RGB')
    pixels = np.asarray(image)
    results = detector.detect_faces(pixels)
    x1, y1, width, height = results[0]['box']
    x1, y1 = abs(x1), abs(y1)
    x2, y2 = x1 + width, y1 + height
    face = pixels[y1:y2, x1:x2]
    image = Image.fromarray(face)
    image = image.resize(required_size)
    face_array = np.asarray(image)
    return face_array

face = extract_face('test.jpeg')

In [6]:
from keras.models import load_model

model = load_model('facenet_keras.h5')

def get_embedding(model, face_pixels):
    face_pixels = face_pixels.astype('float32')
    mean, std = face_pixels.mean(), face_pixels.std()
    face_pixels = (face_pixels - mean) / std
    samples = np.expand_dims(face_pixels, axis=0)
    yhat = model.predict(samples)
    return yhat[0]

embedding = get_embedding(model, face)

FileNotFoundError: [Errno 2] Unable to open file (unable to open file: name = 'facenet_keras.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

In [ ]:
from scipy.spatial.distance import cosine

def compare_embeddings(embedding1, embedding2, threshold=0.5):
    distance = cosine(embedding1, embedding2)
    return distance <= threshold

# Example usage
known_embedding = get_embedding(model, known_face)
new_embedding = get_embedding(model, new_face)

if compare_embeddings(known_embedding, new_embedding):
    print("Match found!")
else:
    print("No match.")

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

image = np.expand_dims(face, axis=0)
it = datagen.flow(image, batch_size=1)
augmented_images = [next(it)[0].astype('uint8') for i in range(10)]

In [ ]:
from sklearn.svm import SVC

# Assuming you have a list of embeddings and corresponding labels
X = np.array(embeddings_list)
y = np.array(labels_list)

model = SVC(kernel='linear', probability=True)
model.fit(X, y)

# Predict new face
new_embedding = get_embedding(model, new_face)
prediction = model.predict([new_embedding])
print(prediction)